In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import time
from datetime import datetime

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)




In [ ]:
weightDecay = 0.0005
momentum = 0.9
learningRate = 0.4
batch_size = 24
L2penalty = weightDecay

In [ ]:
tf.keras.backend.clear_session()

convOptions = {
    "strides": 1,
    "padding": 'SAME', 
    "activation": tf.nn.relu,
    #"kernel_regularizer": regularizers.l2(L2penalty),
    #"bias_regularizer": regularizers.l2(L2penalty),
    "kernel_initializer": tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None),
    #"bias_initalizer": tf.keras.initializers.Zeros()
}


convTransOptions = {
    "strides": (2,2),
    "padding": 'SAME', 
    "activation": tf.nn.relu,
    #"kernel_regularizer": regularizers.l2(L2penalty),
    #"bias_regularizer": regularizers.l2(L2penalty),
    "kernel_initializer": tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None),
    #"bias_initalizer": tf.keras.initializers.Zeros()
}

maxPoolOptions = {
    "pool_size": 2,
    "strides": 2,
    "padding": 'SAME'
}

LInputTarget = tf.keras.Input(dtype = tf.float32, shape = [256, 256, 3], name = 'Target')
layersEncoder = [LInputTarget]
transcoderInputs = []
with tf.name_scope("Encoder"):
    filtersNumber=[64, 64, None, 128, 128, None, 256, 256, None, 512, 512, None, 512, 512, None]
    for fn in filtersNumber:
        if fn is None:
            layersEncoder.append(tf.keras.layers.MaxPool2D(**maxPoolOptions)(layersEncoder[-1]))
            transcoderInputs.append(layersEncoder[-1])
        else:
            layersEncoder.append(tf.keras.layers.Conv2D(filters = fn, kernel_size=3, **convOptions)(layersEncoder[-1]))
    encoderOutput = layersEncoder[-1]
    
    modelEncoder = tf.keras.Model(
        inputs=LInputTarget, 
        outputs=encoderOutput,
        name="Encoder model"
    )

In [ ]:
modelEncoder.summary()

In [ ]:
tf.keras.utils.plot_model(modelEncoder, "unet_encoder.png", show_shapes=True)

In [ ]:
LInputQuery  = tf.keras.Input(dtype = tf.float32, shape = [64, 64, 3], name = 'Query')
layersConditionalEncoder = [LInputQuery]
with tf.name_scope("Conditional"):
    filtersNumber=[32, 32, None, 64, 64, None, 128, None, 256, None, 512, None]
    for fn in filtersNumber:
        if fn is None:
            layersConditionalEncoder.append(
                tf.keras.layers.MaxPool2D(**maxPoolOptions)(layersConditionalEncoder[-1])
            )
        else:
            layersConditionalEncoder.append(
                tf.keras.layers.Conv2D(filters = fn, kernel_size=3, **convOptions)(layersConditionalEncoder[-1])
            )
    layersConditionalEncoder.append(
                tf.keras.layers.Conv2D(filters = 512, 
                                       kernel_size=2, 
                                       strides=2, 
                                       **{k:v for k,v in convOptions.items() if k != 'strides'}
                                      )(layersConditionalEncoder[-1])
            )
    conditionalEncoderOutput = layersConditionalEncoder[-1]

    modelConditional = tf.keras.Model(
        inputs=LInputQuery,
        outputs=conditionalEncoderOutput, 
        name="Latent Representation Encoder"
    )

In [ ]:
modelConditional.summary()

In [ ]:
tf.keras.utils.plot_model(modelConditional, "unet_encoder.png", show_shapes=True)

In [ ]:
with tf.name_scope("Transcoder"):
    layersTransDecoder = []
    upsampledLayers = []
    tiles = [(8,8), (16,16), (32, 32), (64, 64), (128,128)]
    filters = [(None, 512, 512, 512), 
               (512, 512, 512, 512), 
               (256, 256, 256, 256), 
               (128, 128, 128, 128),
               (64, 64, 64, 64)]
    
    for tile, encodedInput, fs in zip(tiles, reversed(transcoderInputs), filters):
        # Tiling output from conditional encoder
        layersTransDecoder.append(tf.keras.layers.UpSampling2D(size=tile)(conditionalEncoderOutput))
        # Concatenating tiled output with reverse order of encoder MaxPool layers
        layersTransDecoder.append(tf.keras.layers.Concatenate()([layersTransDecoder[-1], encodedInput]))
        # Flattening the concatenation with 1x1 conv if needed and joining with last cycle's result
        if fs[0] is not None:
            layersTransDecoder.append(tf.keras.layers.Conv2D(fs[0], kernel_size=1, **convOptions)(layersTransDecoder[-1]))
            layersTransDecoder.append(tf.keras.layers.Concatenate()([layersTransDecoder[-1], upsampledLayers[-1]]))
        # Transdecoding encoded values with Conv2D layers
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[1], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[2], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        # Upsampling with transposed convolution filters, saving the layer for next cycle merging
        layersTransDecoder.append(tf.keras.layers.Conv2DTranspose(fs[3], kernel_size=3, **convTransOptions)(layersTransDecoder[-1]))
        upsampledLayers.append(layersTransDecoder[-1])
        
    unetOutput = tf.keras.layers.Conv2D(filters = 1, kernel_size=3, **convOptions, name="Output")(layersTransDecoder[-1])
    layersTransDecoder.append(unetOutput)

    modelUnet = tf.keras.Model(inputs=[LInputTarget, LInputQuery], outputs=[unetOutput], name="Unet")

In [ ]:
modelUnet.summary()

In [ ]:
print(f'{35659553 * (32/8)/1e6}')


In [ ]:
tf.keras.utils.plot_model(modelUnet, "unet_model.png", show_shapes=True)

# Przygotowanie danych

In [ ]:
import os
MASKS_PATH = '/qarr/studia/magister/datasets/FlickrLogos-v2/classes/masks/'
INPUT_PATH = '/qarr/studia/magister/datasets/FlickrLogos-v2/classes/jpg/'

classes = [o for o in os.listdir(INPUT_PATH) if os.path.isdir(INPUT_PATH + '/' + o)]
classes = [o for o in classes if o != 'no-logo']


In [ ]:
print(classes)

## Wczytywanie obrazków do pamięci

In [ ]:
images = dict()
targets = dict()
queries = dict()
start_time = time.time()

def rescale(nparray, scale=255.0):
    return np.array(nparray, dtype=np.float32)/scale

for c in classes:
    root_input = INPUT_PATH + '/' + c 
    root_masks = MASKS_PATH + '/' + c
    images[c] = list()
    targets[c] = list()
    queries[c] = list()
    
    for f in os.listdir(root_input):
        img = cv2.imread(f'{root_input}/{f}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(f'{root_masks}/{f}.mask.merged.png', cv2.IMREAD_GRAYSCALE)
        bboxes = []
        
        with open(f'{root_masks}/{f}.bboxes.txt') as csvfile:
            bboxread = csv.reader(csvfile, delimiter=' ')
            next(bboxread)
            for row in bboxread:
                bboxes.append(row)
                
        for bbox in bboxes:
            x,y,w,h = [int(i) for i in bbox]
            imgslice = img[y:y+h, x:x+w]
            imgslice = cv2.resize(imgslice, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
            queries[c].append(rescale(imgslice, 255.0))
            # Biore tylko pierwszy z dostepnych bbox na obrazku
            break 
            
        img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        mask = cv2.resize(mask, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    
        images[c].append(rescale(img, 255.0))
        targets[c].append(rescale(mask, 255.0))


end_time = time.time()

print(f'Time taken: {end_time-start_time} seconds')


## Wczytane przykłady obrazków

In [ ]:
for v in images.values():
    plt.imshow(v[0])
    break

In [ ]:
for v in targets.values():
    plt.imshow(v[0], cmap='gist_gray')
    break

In [ ]:
for v in queries.values():
    plt.imshow(v[0])
    break

# Analiza rozmiaru danych

In [ ]:
for c in classes:
    print(f'{c:>12}: {len(images[c])} logos: {len(queries[c]):3<} pairs: {len(images[c])*(len(queries[c])-1)}')
print(f'{"total":<12}: {sum([len(images[c]) for c in classes])} logos: {sum([len(queries[c]) for c in classes])} pairs: {sum([len(images[c])*len(queries[c]) for c in classes])}')

In [ ]:
print(f"{32*(70*69)*(64*64*3+256*256*4)*8/1e9} GB vs {32*(70)*(64*64*3+256*256*4)*8/1e9} GB")

Gdyby brać pod uwagę każde logo z obrazka:
```
      adidas: 70 logos: 120 pairs: 8400
        aldi: 70 logos: 106 pairs: 7420
       apple: 70 logos: 76 pairs: 5320
       becks: 70 logos: 100 pairs: 7000
         bmw: 70 logos: 74 pairs: 5180
   carlsberg: 70 logos: 108 pairs: 7560
      chimay: 70 logos: 112 pairs: 7840
    cocacola: 70 logos: 130 pairs: 9100
      corona: 70 logos: 83 pairs: 5810
         dhl: 70 logos: 123 pairs: 8610
    erdinger: 70 logos: 105 pairs: 7350
        esso: 70 logos: 87 pairs: 6090
       fedex: 70 logos: 94 pairs: 6580
     ferrari: 70 logos: 73 pairs: 5110
        ford: 70 logos: 76 pairs: 5320
     fosters: 70 logos: 98 pairs: 6860
      google: 70 logos: 83 pairs: 5810
     guiness: 70 logos: 98 pairs: 6860
    heineken: 70 logos: 103 pairs: 7210
          hp: 70 logos: 112 pairs: 7840
       milka: 70 logos: 197 pairs: 13790
      nvidia: 70 logos: 114 pairs: 7980
    paulaner: 70 logos: 102 pairs: 7140
       pepsi: 70 logos: 178 pairs: 12460
 rittersport: 70 logos: 204 pairs: 14280
       shell: 70 logos: 96 pairs: 6720
      singha: 70 logos: 83 pairs: 5810
   starbucks: 70 logos: 95 pairs: 6650
stellaartois: 70 logos: 87 pairs: 6090
      texaco: 70 logos: 88 pairs: 6160
    tsingtao: 70 logos: 109 pairs: 7630
         ups: 70 logos: 90 pairs: 6300
total       : 2240 logos: 3404 pairs: 238280
```

# Generator danych

In [ ]:
rnd = np.random.RandomState(13371337)

nclasses = len(classes)
nlogos = sum([len(images[c]) for c in classes])//nclasses
all_cases = nclasses*nlogos*(nlogos-1)
valid_cases = ((all_cases//batch_size)//10)*batch_size
valid_unique_n = 2
valid_unique_pairs = nclasses*2*np.sum(range(nlogos-1, nlogos-valid_unique_n-1, -1))

train_data_permutations = np.zeros((all_cases-valid_cases, 3), dtype=np.int8)
valid_data_permutations = np.zeros((valid_cases, 3), dtype=np.int8)

skips = np.sort(rnd.choice(all_cases-valid_cases, size=valid_cases-valid_unique_pairs+1, replace=False))
skips[-1] = all_cases

trainIt = 0
validIt = 0
skipIt = 0
for c_i in range(nclasses):
    valid_unique = rnd.choice(nlogos, size=valid_unique_n, replace=False)
    for n_i in range(nlogos):
        for l_i in range(nlogos):
            if n_i == l_i:
                continue
            if n_i in valid_unique or l_i in valid_unique:
                valid_data_permutations[validIt] = (c_i, n_i, l_i)
                validIt += 1
            elif skips[skipIt] == trainIt:
                valid_data_permutations[validIt] = (c_i, n_i, l_i)
                validIt += 1
                skipIt += 1
            else:
                train_data_permutations[trainIt] = (c_i, n_i, l_i)                
                trainIt += 1
                
train_data_permutations = rnd.permutation(train_data_permutations)
train_data_permutations = train_data_permutations[:(len(train_data_permutations)//5//batch_size)*batch_size]
valid_data_permutations = rnd.permutation(valid_data_permutations)


In [ ]:
def describe(x):
    try:
        return f'{x.shape}'
    except AttributeError:
        return f"{'[' + ', '.join([describe(q) for q in x]) + ']'}"

describe(train_data_permutations)
print(all_cases)

In [ ]:
def dataset_permutations_generator(batch_size, data_permutations, repeat=True, shuffle=True):
    s = 0
    outimage = []
    outquery = []
    outtarget = []
    loop = True
    while loop:
        if shuffle:
            data_permutations = np.random.permutation(data_permutations)
        for class_number, image_number, query_number in data_permutations:
            c = classes[class_number]
            outimage.append(images[c][image_number])
            outquery.append(queries[c][query_number])
            outtarget.append(targets[c][image_number])
            s += 1
            if s >= batch_size:
                s = 0
                yield (np.reshape(outimage, (batch_size, 256, 256, 3)),
                       np.reshape(outquery, (batch_size, 64, 64, 3))
                      ), np.reshape(outtarget, (batch_size, 256, 256, 1))
                outimage = []
                outquery = []
                outtarget = []
    loop = repeat

## Test generatora

In [ ]:
testgen = dataset_permutations_generator(1, train_data_permutations, shuffle=False)
tdat = next(testgen)

In [ ]:
plt.imshow(tdat[0][0][0])

In [ ]:
batch_size = 1
unetDataset = tf.data.Dataset.from_generator(dataset_permutations_generator,
                                             args=[batch_size, train_data_permutations],
                                             output_types=((tf.float32, tf.float32), tf.float32),
                                             output_shapes=(((batch_size, 256,256,3), (batch_size, 64,64,3)),
                                                          (batch_size, 256,256,1))
                                            )
#unetDataset = unetDataset.cache()

# Zestawienie modelu i uczenie - próba

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate, momentum=momentum, nesterov=False, name="SGD") # weight decay 0.0005 by L2

modelUnet.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.BinaryCrossentropy(
                        name="binary_crossentropy")]
                 )

# Próba generalna

In [ ]:
batch_size = 24

In [ ]:
unetValidDataset = tf.data.Dataset.from_generator(dataset_permutations_generator,
                                             args=[batch_size, valid_data_permutations],
                                             output_types=((tf.float32, tf.float32), tf.float32),
                                             output_shapes=(((batch_size, 256,256,3), (batch_size, 64,64,3)),
                                                          (batch_size, 256,256,1))
                                            )

In [ ]:
unetTrainDataset = tf.data.Dataset.from_generator(dataset_permutations_generator,
                                             args=[batch_size, train_data_permutations],
                                             output_types=((tf.float32, tf.float32), tf.float32),
                                             output_shapes=(((batch_size, 256,256,3), (batch_size, 64,64,3)),
                                                          (batch_size, 256,256,1))
                                            )

In [ ]:
# Create a TensorBoard callback
logs = "/qarr/studia/magister/models/logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

try:
    tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1)
                                                 #profile_batch = '1,3')
except AlreadyExistsError:
    print("Already exists, skipping")

In [ ]:
callbackCheckpoint = tf.keras.callbacks.ModelCheckpoint(
    "/qarr/studia/magister/models/" + datetime.now().strftime("%Y%m%d-%H%M%S"),
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch",
    options=None
)

In [ ]:
#del(modelUnet)

In [ ]:
modelUnet.fit(unetTrainDataset, 
              epochs=2, 
              steps_per_epoch=len(train_data_permutations)//batch_size, 
              validation_data=unetValidDataset,
              validation_steps=valid_cases//batch_size,
              #callbacks=[tboard_callback]#, callbackCheckpoint]
             ) # batch_size unspecified since it's generated by generator

In [ ]:
modelUnet.save('unet_test_model_small_02.model')

# Analiza wyników na przykładach

In [ ]:
example = unetTrainDataset.take(1)

In [ ]:
example_result = modelUnet.predict(example)
example = list(example.as_numpy_iterator())

In [ ]:
subs = plt.subplots(1,4)
subs = subs[0].axes
subs[0].imshow(example[0][0][1][0])
subs[1].imshow(example[0][0][0][0])
subs[2].imshow(np.reshape(example[0][1][0], (256,256)), cmap='gist_gray')
subs[3].imshow(np.reshape(example_result[0], (256,256)), cmap='gist_gray')

# Wczytanie zapisanej sieci i kompilacja

In [ ]:
modelUnet = tf.keras.models.load_model("unet_test_model03.model")

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate, momentum=momentum, nesterov=False, name="SGD") # weight decay 0.0005 by L2

modelUnet.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(
    from_logits=False, label_smoothing=0, reduction="auto", name="binary_crossentropy"
),
              #loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.BinaryCrossentropy(
                        name="binary_crossentropy")]
                 )

# Nowy callback do logowania gradientu

In [ ]:
class ExtendedTensorBoard(tf.keras.callbacks.TensorBoard):
    def _log_gradients(self, epoch):
        writer = self._get_writer(self._train_run_name)

        with writer.as_default(), tf.GradientTape() as g:
            # here we use test data to calculate the gradients
## Wrong approach dont do that, use internal variables of actual data not external lists
            features, y_true = list(val_dataset.batch(100).take(1))[0]

            y_pred = self.model(features)  # forward-propagation
            loss = self.model.compiled_loss(y_true=y_true, y_pred=y_pred)  # calculate loss
            gradients = g.gradient(loss, self.model.trainable_weights)  # back-propagation

            # In eager mode, grads does not have name, so we get names from model.trainable_weights
            for weights, grads in zip(self.model.trainable_weights, gradients):
                tf.summary.histogram(
                    weights.name.replace(':', '_') + '_grads', data=grads, step=epoch)

        writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        # This function overwrites the on_epoch_end in tf.keras.callbacks.TensorBoard
        # but we do need to run the original on_epoch_end, so here we use the super function.
        super(ExtendedTensorBoard, self).on_epoch_end(epoch, logs=logs)

        if self.histogram_freq and epoch % self.histogram_freq == 0:
            self._log_gradients(epoch)

In [ ]:
class ExtendedTensorBoard(tf.keras.callbacks.TensorBoard):
  def _log_gradients(self, epoch):
    step = tf.cast(tf.math.floor((epoch+1)*num_instance/batch_size), dtype=tf.int64)
    writer = self._get_writer(self._train_run_name)

    with writer.as_default(), tf.GradientTape() as g:
      # here we use test data to calculate the gradients
      _x_batch = x_te[:100]
      _y_batch = y_te[:100]

      g.watch(_x_batch)
      _y_pred = self.model(_x_batch)  # forward-propagation
      loss = self.model.loss(y_true=_y_batch, y_pred=_y_pred)  # calculate loss
      gradients = g.gradient(loss, self.model.trainable_weights)  # back-propagation

      # In eager mode, grads does not have name, so we get names from model.trainable_weights
      for weights, grads in zip(self.model.trainable_weights, gradients):
        tf.summary.histogram(
            weights.name.replace(':', '_')+'_grads', data=grads, step=step)
    
    writer.flush()

  def on_epoch_end(self, epoch, logs=None):  
    # This function overwrites the on_epoch_end in tf.keras.callbacks.TensorBoard
    # but we do need to run the original on_epoch_end, so here we use the super function. 
    super(ExtendedTensorBoard, self).on_epoch_end(epoch, logs=logs)

    if self.histogram_freq and epoch % self.histogram_freq == 0:
      self._log_gradients(epoch)

In [ ]:
# Define the Gradient Function
epoch_gradient = []

def get_gradient_func(model):
    grads = tf.keras.backend.gradients(model.total_loss, model.trainable_weights)
    # grads = K.gradients(model.loss, model.trainable_weights)
    # inputs = model.model.inputs + model.targets + model.sample_weights
    # use below line of code if above line doesn't work for you
    # inputs = model.model._feed_inputs + model.model._feed_targets + model.model._feed_sample_weights
    inputs = model._feed_inputs + model._feed_targets + model._feed_sample_weights
    func = tf.keras.backend.function(inputs, grads)
    return func

# Define the Required Callback Function
class GradientCalcCallback(tf.keras.callbacks.TensorBoard):
    def on_epoch_end(self, epoch, logs=None):
        super(ExtendedTensorBoard, self).on_epoch_end(epoch, logs=logs)
        if self.histogram_freq and epoch % self.histogram_freq == 0:
            self._log_gradients(epoch)
        get_gradient = get_gradient_func(model)
        grads = get_gradient([test_images, test_labels, np.ones(len(test_labels))])
        
        epoch_gradient.append(grads)


In [ ]:
# Define the Gradient Function
epoch_gradient = []

def get_gradient_func(model):
    grads = K.gradients(model.total_loss, model.trainable_weights)
    # grads = K.gradients(model.loss, model.trainable_weights)
    # inputs = model.model.inputs + model.targets + model.sample_weights
    # use below line of code if above line doesn't work for you
    # inputs = model.model._feed_inputs + model.model._feed_targets + model.model._feed_sample_weights
    inputs = model._feed_inputs + model._feed_targets + model._feed_sample_weights
    func = K.function(inputs, grads)
    return func

# Define the Required Callback Function
class GradientCalcCallback(tf.keras.callbacks.TensorBoard):
  def on_epoch_end(self, epoch, logs=None):
      get_gradient = get_gradient_func(model)
      grads = get_gradient([test_images, test_labels, np.ones(len(test_labels))])
      epoch_gradient.append(grads)

In [ ]:
def gradient(model, x_tensor):
    #x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)
    with tf.GradientTape() as t:
        t.watch(x_tensor)
        loss = model(x_tensor)
    return t.gradient(loss, x_tensor).numpy()

In [ ]:
loss = modelUnet.evaluate(tdat)
describe(tdat)

In [ ]:
tt = tf.Tensor(tdat[0])

with tf.GradientTape() as tape:
    tape.watch(tt)

In [ ]:
gradient(modelUnet, unetDataset.take(1))

In [ ]:
modelUnet(tdat)